# Demo Notebook for Creating a Reboot Motion Inverse Kinematics Report

Run the cells in order, making sure to enter AWS credentials in the cell when prompted


In [ ]:
#@title Install Python Package

!pip install -q git+https://github.com/RebootMotion/colab-notebook-utils.git@v1.0.0#egg=colab_notebook_utils

In [ ]:
import awswrangler as wr
import boto3
import colab_notebook_utils as utils
import matplotlib.pyplot as plt
import os

from colab_notebook_utils import S3Metadata, MocapType, MovementType, Handedness, FileType, PlayerMetadata

In [ ]:
#@title Org and AWS Credentials

from dotenv import load_dotenv

load_dotenv()

if 'AWS_ACCESS_KEY_ID' not in os.environ:
    from getpass import getpass

    os.environ['ORG_ID'] = getpass('Enter reboot-motion org_id here:')
    os.environ['AWS_ACCESS_KEY_ID'] = getpass('Enter AWS_ACCESS_KEY_ID here:')
    os.environ['AWS_SECRET_ACCESS_KEY'] = getpass('Enter SECRET_ACCESS_KEY here:')
    os.environ['AWS_DEFAULT_REGION'] = getpass('Enter AWS_DEFAULT_REGION here:')

boto3_session = boto3.session.Session(
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    region_name=os.environ['AWS_DEFAULT_REGION']
    )

print('Current Boto3 Session:')
print(boto3_session)

In [ ]:
#@title Set S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_type=MocapType.HAWKEYE_HFR,
    movement_type=MovementType.BASEBALL_PITCHING,
    handedness=Handedness.LEFT,
    file_type=FileType.INVERSE_KINEMATICS
)

s3_df = utils.download_s3_summary_df(s3_metadata)

In [ ]:
#@title Set Primary Analysis Segment Info

primary_segment_label = '2023-04-13'

primary_analysis_segment = PlayerMetadata(
    mlbam_player_ids=['605488'],
    session_dates=None,
    game_pks=None, # None defaults to all
    session_date_start = '2023-04-13',
    session_date_end = None,
    year = None,
    mlb_play_guid=None, # set the mlb play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=s3_metadata,
)

primary_segment_summary_df = utils.filter_s3_summary_df(primary_analysis_segment, s3_df)

In [ ]:
#@title List all Available S3 data for the Primary Analysis Segment

available_s3_keys = utils.list_available_s3_keys(os.environ['ORG_ID'], primary_segment_summary_df)

In [ ]:
#@title Load the Primary Analysis Segment into an Analysis Dictionary with Mean and Standard Dev DataFrames

primary_segment_data_df = utils.load_games_to_df_from_s3_paths(primary_segment_summary_df['s3_path_delivery'].tolist())

primary_segment_dict = utils.load_data_into_analysis_dict(primary_analysis_segment, primary_segment_data_df, segment_label=primary_segment_label)

In [ ]:
#@title Optional - Uncomment below and set Comparison Analysis Segment Inputs

comparison_segment_label = 'Early 2023'

comparison_s3_metadata = s3_metadata

comparison_analysis_segment = PlayerMetadata(
    mlbam_player_ids=['605488'],
    session_dates=None, # None defaults to all
    game_pks=None, # None defaults to all,
    session_date_start = None,
    session_date_end = '2023-04-12',
    year = None,
    mlb_play_guid=None, # set the mlb play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=comparison_s3_metadata,
)

comparison_segment_summary_df = utils.filter_s3_summary_df(comparison_analysis_segment, s3_df)

comparison_segment_data_df = utils.load_games_to_df_from_s3_paths(comparison_segment_summary_df['s3_path_delivery'].tolist())

comparison_segment_dict = utils.load_data_into_analysis_dict(comparison_analysis_segment, comparison_segment_data_df, segment_label=comparison_segment_label)

In [ ]:
#@title Put One or Two Analysis Dicts into a list to be sent to AWS for Analysis

# analysis_dicts = [primary_segment_dict]
analysis_dicts = [primary_segment_dict, comparison_segment_dict]

In [ ]:
#@title Optional - Create Simple Comparison Plots of Joint Angles

x_column = 'time_from_max_hand'  # 'time_from_max_hand', 'norm_time', 'rel_fame', 'time'

angle_names = ['pelvis_rot', 'pelvis_side', 'torso_rot', 'torso_side', 'torso_ext', 'left_shoulder_rot', 'left_shoulder_flex', 'left_shoulder_abd', 'left_elbow']

stand_devs_to_shade = 1.0

figs = []

for angle_name in angle_names:

    fig = plt.figure()

    for segment_dict in analysis_dicts:

        y = segment_dict['df_mean'][angle_name]
        y_lo = segment_dict['df_mean'][angle_name] - (stand_devs_to_shade * segment_dict['df_std'][angle_name])
        y_hi = segment_dict['df_mean'][angle_name] + (stand_devs_to_shade * segment_dict['df_std'][angle_name])

        plt.fill_between(segment_dict['df_mean'][x_column], y_lo, y_hi, alpha=0.4)
        plt.plot(segment_dict['df_mean'][x_column], y, label=segment_dict['segment_label'])

    # plt.xlim([-1, 0.1])  # uncomment to limit the width of the x-axis
    plt.legend()
    plt.grid()

    plt.ylabel("joint angle (deg)")
    plt.xlabel(x_column)

    plt.title(angle_name)

    figs.append(fig)

    plt.show()


In [ ]:
#@title Optional - Save Plots to a PDF

from matplotlib.backends.backend_pdf import PdfPages

pdf_file_name = 'analysis.pdf'

pdf_analysis = PdfPages(pdf_file_name)

for fig in figs:

    pdf_analysis.savefig(fig)

pdf_analysis.close()

print('Saved plots to', pdf_file_name)

In [ ]:
#@title Get Population Inverse Kinematics Data from S3

print('Loading data from:', s3_metadata.s3_population_prefix)

print('Downloading population mean...')
pop_mean_df = wr.s3.read_csv([f"{s3_metadata.s3_population_prefix}mean_ik.csv"], index_col=[0])

print('Downloading population standard deviation...')
pop_std_df = wr.s3.read_csv([f"{s3_metadata.s3_population_prefix}std_ik.csv"], index_col=[0])

print('Done!')

In [ ]:
#@title Inspect Available Joint Angle Names

joint_angle_names = utils.get_available_joint_angles(analysis_dicts)
print(f"Available Joint Angles:\nn={len(joint_angle_names)}\n{joint_angle_names}")

In [ ]:
#@title Set Plot Information for Synchronized Animation

time_column_to_plot = 'time_from_max_hand'  # seconds from max dom hand velo

joint_angles_to_plot = ['torso_side', 'left_elbow']  # list of joint angles to plot below, from available angles above

# set to True to plot the mean joint angle trace across the selection,
# set to False to plot the joint angle trace for the play specified by the mlb_play_guid in the analysis_dict from that cell above
plot_joint_angle_mean = True

# set to True to write an html file for each joint angle skeleton animation
write_individual_html = False

# step size between animation frames
frame_step = 25

In [ ]:
#@title Create Animation by Sending Plot Information to AWS

figs = []

for joint_angle_to_plot in joint_angles_to_plot:
    fig = utils.get_animation(boto3_session, analysis_dicts, pop_mean_df, pop_std_df, time_column_to_plot, joint_angle_to_plot, plot_joint_angle_mean, frame_step=frame_step)
  
    if write_individual_html:
        fig.write_html(f'{joint_angle_to_plot}_animation.html', full_html=True, include_plotlyjs='cdn')

    figs.append(fig)

    fig.show()

In [ ]:
#@title Create Joint Angle HTML Plots
import numpy as np

joint_angle_names_no_zeros = [
    angle for angle in joint_angle_names 
    if not np.allclose(pop_mean_df[angle].values, np.zeros(len(pop_mean_df)))
    ]

joint_angle_lists = utils.list_chunks(joint_angle_names_no_zeros, 3)

plot_colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
               'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
               'rgb(148, 103, 189)']

time_label = 'time_from_max_hand'
figs_angles = []

for joint_angle_list in joint_angle_lists:

    fig = utils.get_joint_plot(boto3_session, analysis_dicts, pop_mean_df, pop_std_df, time_label, joint_angle_list)
    fig.show()
    figs_angles.append(fig)

In [ ]:
#@title Write Report HTML to Local CoLab Folder
utils.save_figs_to_html(figs+figs_angles)